<a href="https://colab.research.google.com/github/hoidv/CS114.P11/blob/main/create_split_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
root_directory = '/content/drive/MyDrive/Public'

In [4]:
def check_valid_and_parse_filename(filename):
    #Kiểm tra đuôi file
    name, extension = os.path.splitext(filename)
    if extension not in [".jpg", ".jpeg", ".png"]:
        return None, None, None

    #Tách theo dấu "."
    parts = name.split(".")
    if len(parts) == 3:
        mssv_part, brand, order = parts[0], parts[1], parts[2]
        contributions = mssv_part.split("-")

        #Kiểm tra kiểu dữ liệu mssv, stt
        num_pattern = r"^\d+$"
        if re.match(num_pattern, order) == None:
          return None, None, None
        for contribute in contributions:
          if re.match(num_pattern, contribute) == None:
            return None, None, None

        return contributions, brand, order
    return None, None, None

In [16]:
import csv
import re
import os

cars_brand_set = {'Others': 0,
                  'Honda' : 1,
                  'Hyundai' : 2,
                  'KIA' : 3,
                  'Mazda' : 4,
                  'Mitsubishi' : 5,
                  'Suzuki' : 6,
                  'Toyota' : 7,
                  'VinFast': 8}

colab_path = '/content/drive/MyDrive/Colab_Notebooks/data'
os.makedirs(colab_path, exist_ok=True)
data_path = os.path.join(colab_path, "CarDataset.csv")

with open(data_path, mode='w', newline='', encoding='utf-8') as file:
  writer = csv.writer(file)
  writer.writerow(["Path", "CategoryID"])
  for car_brand in cars_brand_set.keys():
    brand_path = os.path.join(root_directory, car_brand)
    for file in os.listdir(brand_path):
        contributions, brand, order = check_valid_and_parse_filename(file)
        if contributions != None:

          #Kiểm tra brand trong tên file có giống theo định dạng brand trong thư mục không
          brand_pattern = rf"^{car_brand}$"
          match_brand = re.match(brand_pattern, brand)
          if match_brand != None:
            writer.writerow([f"{car_brand}/{file}", str(cars_brand_set[car_brand])])

In [15]:
!ls {colab_path}

CarDataset.csv


In [18]:
import pandas as pd
all_brand_df = pd.read_csv(data_path)

#Tạo từ điển lưu DataFrame cho từng car_brand
brand_data = {}
for car_brand in cars_brand_set:
  brand_data[car_brand] = all_brand_df[all_brand_df['CategoryID'] == cars_brand_set[car_brand]]

In [19]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)

#Tạo X_train gồm 5 dict mỗi dict là một fold
X_train = [dict() for _ in range(5)]
X_val = [dict() for _ in range(5)]

#Duyệt qua từng hãng xe tạo fold cho từng hiệu xe
for car_brand in cars_brand_set.keys():
  for i, (train_index, val_index) in enumerate(kf.split(brand_data[car_brand])):
    X_train[i][car_brand] = train_index
    X_val[i][car_brand] = val_index


In [20]:
#Duyệt qua 5 fold
import os

colab_path = '/content/drive/MyDrive/Colab Notebooks/data'
os.makedirs(colab_path, exist_ok=True)

for fold_num in range(5):

  #Tạo file CarDataset_train
  train_indices_by_brand = X_train[fold_num]
  lst_brand_train_df = [brand_data[brand].iloc[train_indices_by_brand[brand]] for brand in cars_brand_set.keys()]
  split_train_df = pd.concat(lst_brand_train_df, ignore_index=True)
  split_train_path = os.path.join(colab_path, f"CarDataset-Splits-{fold_num+1}-train.csv")
  split_train_df.to_csv(split_train_path, index=False)

  #Tạo file CarDataset_val
  val_indices_by_brand = X_val[fold_num]
  lst_brand_val_df = [brand_data[brand].iloc[val_indices_by_brand[brand]] for brand in cars_brand_set.keys()]
  split_val_df = pd.concat(lst_brand_val_df, ignore_index=True)
  split_val_path = os.path.join(colab_path, f"CarDataset-Splits-{fold_num+1}-val.csv")
  split_val_df.to_csv(split_val_path, index=False)